In [17]:
import time
import yaml
import os
from collections import deque

from apwf.tools import fx_comp_helper, fx_daq_helper, executor_helper 
from apwf.flows import automate_helper

from funcx.sdk.client import FuncXClient
from globus_automate_client import create_flows_client


configfile = f"{os.getcwd()}/ptycho_wf_config.yml"
wf_config= yaml.safe_load(open(configfile))

apwfLogger = executor_helper.setup_logger(wf_config)

In [25]:
# Globus Online Endpoints
src_endpoint = wf_config['globus_endpoints']['aps_mona4_gcp']
dest_endpoint = wf_config['globus_endpoints']['alcf_theta']

# FuncX endpoint at ThetaGPU (ALCF) and Prisma (APS)
compute_fx_endpoint = wf_config['funcx_endpoints']['alcf_polaris_gpu']
daq_fx_endpoint = wf_config['funcx_endpoints']['aps_mona4_daq']

print(compute_fx_endpoint)

b35e121c-5ed6-4980-a32e-9aee09089c36


In [26]:
fxc = FuncXClient()

In [27]:
# Register setup functions to funcx service
daq_wf_fxid = fxc.register_function(fx_daq_helper.daq_wf_prepare)
comp_wf_fxid = fxc.register_function(fx_comp_helper.comp_wf_prepare)

## Prepare workflow environment ##
# Executor
res = executor_helper.executor_wf_prepare(wf_config['executor']['activated_iids_file_path'],
                                          wf_config['executor']['log_folder_path'],
                                          create_activated_iid_file=True, create_log_folder=True, 
                                          cleanup_activated_iid_file=True, cleanup_log_folder=False)
if res != 0: apwfLogger.error(f"Unable to setup executor folders/files: {res}")
else: apwfLogger.info("Executor folders/files are ready.")

# Data acquisition
rid = fxc.run(f"{wf_config['flow']['source']['root_folder_path']}/{wf_config['flow']['source']['input_folder_prefix']}",
              f"{wf_config['flow']['source']['root_folder_path']}/{wf_config['flow']['source']['input_position_folder_prefix']}",
              f"{wf_config['flow']['source']['root_folder_path']}/{wf_config['flow']['source']['output_folder_prefix']}",
              probe_file_path=None, create_output_folder=True, cleanup_output_folder=False,
              endpoint_id=daq_fx_endpoint, function_id=daq_wf_fxid)
res = automate_helper.fx_get_result(rid, fxc)
if res != 0: apwfLogger.error(f"Unable to setup data acquisition machine folders/files: {res}")
else: apwfLogger.info("Data acquisition machine's folders/files are ready.")

# Compute machine
rid = fxc.run(f"{wf_config['flow']['destination']['root_folder_path']}/{wf_config['flow']['destination']['input_folder_prefix']}",
              f"{wf_config['flow']['destination']['root_folder_path']}/{wf_config['flow']['destination']['output_folder_prefix']}",
              f"{wf_config['flow']['task']['executable']['params']['log_folder_path']}",
              create_input_folder=True, create_output_folder=True, create_log_folder=True, 
              cleanup_input_folder=True, cleanup_output_folder=True, cleanup_log_folder=True,
              endpoint_id=compute_fx_endpoint, function_id=comp_wf_fxid)
res = automate_helper.fx_get_result(rid, fxc)
if res != 0: apwfLogger.error(f"Unable to setup compute machine folders/files: {res}")
else: apwfLogger.info("Compute machines' folders/files are ready.")


2022-05-17 08:41:21 INFO Executor folders/files are ready.
2022-05-17 08:41:21 INFO Executor folders/files are ready.
2022-05-17 08:41:21 INFO Executor folders/files are ready.


INFO:apwf:Executor folders/files are ready.


2022-05-17 08:41:24 INFO Data acquisition machine's folders/files are ready.
2022-05-17 08:41:24 INFO Data acquisition machine's folders/files are ready.
2022-05-17 08:41:24 INFO Data acquisition machine's folders/files are ready.


INFO:apwf:Data acquisition machine's folders/files are ready.


2022-05-17 08:41:34 INFO Compute machines' folders/files are ready.
2022-05-17 08:41:34 INFO Compute machines' folders/files are ready.
2022-05-17 08:41:34 INFO Compute machines' folders/files are ready.


INFO:apwf:Compute machines' folders/files are ready.


In [28]:
# FuncX function definition for remote Polaris GPU reconstruction call
func_ptycho_uuid = fxc.register_function(fx_comp_helper.ptycho_func)
#func_ptycho_uuid = fxc.register_function(fx_comp_helper.ptycho_funcx_gpu_pin)
print(func_ptycho_uuid)

cdc6d5ed-7d8a-4069-8718-2184cbc0cc30


In [29]:
# FuncX function definition for remote DAQ machine directory calls
fx_func_get_file_paths_uuid = fxc.register_function(fx_daq_helper.get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(fx_daq_helper.get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

da5f14b9-891c-48a8-b9ed-2a19f04fe62c
bec6256d-f632-48bf-b127-c18c17f0b177


In [30]:
# Globus Automate flow definition
flow_definition = automate_helper.ptycho_flow_definition

# Create a flow with the above flow definition
flows_client = create_flows_client()
flow = flows_client.deploy_flow(flow_definition, input_schema={}, title=wf_config['flow']['title'], keywords=wf_config['flow']['keywords'])
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

In [31]:
src_config = wf_config['flow']['source']
dest_config = wf_config['flow']['destination']
task_config = wf_config['flow']['task']
exec_params_config = task_config['executable']['params']

wfc = executor_helper.EConfigs(src_config, dest_config, wf_config['flow']['only_larger_than_scanid'])

while True:
    # Get the remote folders at source input folder
    src_input_folder_paths_regex = f"{wfc.src_wf_root_path}/{wfc.src_input_folder_prefix}/*"
    rid = fxc.run(src_input_folder_paths_regex,
                  endpoint_id=daq_fx_endpoint,
                  function_id=fx_func_get_folder_paths_uuid)
    src_input_folder_paths = automate_helper.fx_get_result(rid, fxc, delay=wf_config['funcx_req_delay'])

    # Get the remote folders at source output folder
    src_output_folder_paths_regex = f"{wfc.src_wf_root_path}/{wfc.src_output_folder_prefix}/*"
    rid = fxc.run(src_output_folder_paths_regex,
                  endpoint_id=daq_fx_endpoint,
                  function_id=fx_func_get_folder_paths_uuid)
    ex_src_output_folder_paths = automate_helper.fx_get_result(rid, fxc, delay=wf_config['funcx_req_delay'])

    # Get the activated iids and find the iids that need to be processed
    activated_iids = executor_helper.read_activated_iids(wf_config['executor']['activated_iids_file_path'])
    diff_iids = executor_helper.get_unprocessed_iids(activated_iids, src_input_folder_paths, ex_src_output_folder_paths, wfc.add_larger_than)
    
    # If there is no iid/scan to be processed sleep, and then re-check 
    if(len(diff_iids)==0):
        apwfLogger.debug(f"There is no folder to be processed, "
                      f"sleeping {wf_config['data_generation_delay']} secs.")
        time.sleep(wf_config['data_generation_delay'])
        continue
    
    # There are scans to be processed, generate paths
    paths = executor_helper.WFPaths(wfc, src_input_folder_paths, diff_iids).plist
     
    # Generate flow inputs according to the paths and other configuration info.
    flow_inputs = executor_helper.WFFlowsInputs(src_endpoint, dest_endpoint, compute_fx_endpoint, 
                                            func_ptycho_uuid, task_config, exec_params_config, 
                                            wf_config['flow']['sample_name'], paths).filist

    ### Initiate/dispatch Flows ###
    q1 = deque()
    for i in range(len(flow_inputs)):
        label = str(flow_inputs[i]['iid']) + '-' + "-".join(wf_config['flow']['keywords'])
        flow_action = flows_client.run_flow(flow_id, flow_scope, flow_inputs[i], label=label)
        q1.append(flow_action)
        executor_helper.append_activated_iids([flow_inputs[i]['iid']], wf_config['executor']['activated_iids_file_path'])
        apwfLogger.info(f"{i}: Flow {flow_inputs[i]['iid']} initiated and added to q1: {flow_action['action_id']}")

Please log into Globus here:
----------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=e6c75d97-532a-4c88-b031-8584a319fa3e&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F6f00f44e-6ef0-4c75-900c-892e20fd82ce%2Fflow_6f00f44e_6ef0_4c75_900c_892e20fd82ce_user&state=_default&response_type=code&code_challenge=ng4v_N88-anmImk26EXmFDbpoNhkr9heMrwarn5jPg0&code_challenge_method=S256&access_type=offline&prefill_named_grant=Globus+Automate+Command+Line+Interface+on+hostel.xray.aps.anl.gov
----------------------------

Enter the resulting Authorization Code here:2022-05-17 08:47:00 INFO 0: Flow 1702 initiated and added to q1: bce9b06b-97f3-48fc-8a99-a7b2c4031648
2022-05-17 08:47:00 INFO 0: Flow 1702 initiated and added to q1: bce9b06b-97f3-48fc-8a99-a7b2c4031648
2022-05-17 08:47:00 INFO 0: Flow 1702 initiated and added to q1: bce9b06b-97f3-48fc-8a99-a7b2c4031648


INFO:apwf:0: Flow 1702 initiated and added to q1: bce9b06b-97f3-48fc-8a99-a7b2c4031648


2022-05-17 08:47:06 INFO 0: Flow 1706 initiated and added to q1: ee3952af-0118-45b1-9c18-b19266150409
2022-05-17 08:47:06 INFO 0: Flow 1706 initiated and added to q1: ee3952af-0118-45b1-9c18-b19266150409
2022-05-17 08:47:06 INFO 0: Flow 1706 initiated and added to q1: ee3952af-0118-45b1-9c18-b19266150409


INFO:apwf:0: Flow 1706 initiated and added to q1: ee3952af-0118-45b1-9c18-b19266150409


2022-05-17 08:47:07 INFO 1: Flow 1705 initiated and added to q1: 3bb9824e-ed86-45b3-b400-2c13de8c6ea9
2022-05-17 08:47:07 INFO 1: Flow 1705 initiated and added to q1: 3bb9824e-ed86-45b3-b400-2c13de8c6ea9
2022-05-17 08:47:07 INFO 1: Flow 1705 initiated and added to q1: 3bb9824e-ed86-45b3-b400-2c13de8c6ea9


...

2022-05-17 08:54:43 INFO 0: Flow 1778 initiated and added to q1: 840710aa-b9d7-41ff-ac68-c12e9df23f42
2022-05-17 08:54:43 INFO 0: Flow 1778 initiated and added to q1: 840710aa-b9d7-41ff-ac68-c12e9df23f42
2022-05-17 08:54:43 INFO 0: Flow 1778 initiated and added to q1: 840710aa-b9d7-41ff-ac68-c12e9df23f42


INFO:apwf:0: Flow 1778 initiated and added to q1: 840710aa-b9d7-41ff-ac68-c12e9df23f42


2022-05-17 08:54:49 INFO 0: Flow 1779 initiated and added to q1: 63485022-1b4d-41b6-a49c-57dc5c50e407
2022-05-17 08:54:49 INFO 0: Flow 1779 initiated and added to q1: 63485022-1b4d-41b6-a49c-57dc5c50e407
2022-05-17 08:54:49 INFO 0: Flow 1779 initiated and added to q1: 63485022-1b4d-41b6-a49c-57dc5c50e407


INFO:apwf:0: Flow 1779 initiated and added to q1: 63485022-1b4d-41b6-a49c-57dc5c50e407


TypeError: 'NoneType' object is not iterable

In [ ]:
# Cancel all flows
for flow_action in q1:
    flows_client.flow_action_cancel(flow_id, flow_scope, flow_action['action_id'])